In [ ]:
import base64
import json
import uuid

import boto3
import requests


class AwsClientFactory:
    def __init__(self, role_arn):
        self.role_arn = role_arn
        
    def _create_session(self, role_arn):
        client = boto3.client("sts")

        response = client.assume_role(RoleArn=role_arn, RoleSessionName=str(uuid.uuid1()))

        return boto3.session.Session(
            aws_access_key_id=response["Credentials"]["AccessKeyId"],
            aws_secret_access_key=response["Credentials"]["SecretAccessKey"],
            aws_session_token=response["Credentials"]["SessionToken"],
        )
    
    def get_client(self, client_name):
        session = self._create_session(self.role_arn)
        return session.client(client_name)
        
    
class AwsSecretsManagerHelper:
    def __init__(self, role_arn):
        self.role_arn = role_arn
        client_factory = AwsClientFactory(role_arn)
        self.secrets_client = client_factory.get_client('secretsmanager')
        
    def get_secret(self, secret_id):
        response = self.secrets_client.get_secret_value(
            SecretId=secret_id
        )

        return response['SecretString']

class StacksApi:
    requests_url = "https://stacks.api.wellcomecollection.org/stacks/v1/requests"
    works_url = "https://stacks.api.wellcomecollection.org/stacks/v1/works"
        
    def __init__(self, role_arn, client_id, api_key, username, password):
        self.cognito_client_id = client_id
        self.stacks_api_key = api_key
        
        client_factory = AwsClientFactory(role_arn)
        self.cognitoidp_client = client_factory.get_client('cognito-idp')
        self.access_token = self.get_access_token(username, password)

    # Authentication

    def _process_auth_response(self, response):
        if 'AuthenticationResult' in response:
            auth_result = response['AuthenticationResult']
            if 'AccessToken' in auth_result:
                return auth_result
            else:
                print(f"No access token in result: {auth_result}")

        else:
            print(f"Unknown auth response: {response}")
            raise 

    def get_access_token(self, username, password):
        response = self.cognitoidp_client.initiate_auth(
            AuthFlow="USER_PASSWORD_AUTH",
            AuthParameters={
                'USERNAME' : username,
                'PASSWORD' : password,
            },
            ClientId=self.cognito_client_id
        )

        access_token = self._process_auth_response(response)

        return access_token['IdToken']

    #  /works

    def get_work_cors(self):
        response = requests.options(
            f"{StacksApi.works_url}/foo",
            headers={"x-api-key": self.stacks_api_key},
        )

        return response.headers
    
    def get_work(self, work_id):
        response = requests.get(
            f"{StacksApi.works_url}/{work_id}",
            headers={"x-api-key": self.stacks_api_key},
        )

        return json.loads(response.content)

    # /requests

    def _create_item_request(self, item_id):
        return {
          "item": {
            "id": item_id,
            "type": "Item"
          },
          "pickupDate": "2020-03-03T00:00:00Z",
          "type": "Request"
        }
    
    def request_item(self, item_id):        
        response = requests.post(
            StacksApi.requests_url,
            json = self._create_item_request(item_id),
            headers={"Authorization": self.access_token},
        )
        
        content = {}
            
        if(response.content):
            content = json.loads(response.content)
        
        return {
            'status_code': response.status_code,
            'content': content
        }
        
    def list_requests(self):
        response = requests.get(
            StacksApi.requests_url,
            headers={"Authorization": self.access_token},
        )

        return json.loads(response.content)

    
class SierraApi:
    def __init__(self, username, password):
        self.api_token = self._get_token(username, password)
    
    def _get_field(self, varFields, fieldId):
        filteredVarFields = [d for d in varFields if d["fieldTag"] == fieldId]

        field = None
        if filteredVarFields:
            field = filteredVarFields[0].get("content", None)

        return field

    def _get_token(self, username, password):
        encoded = base64.b64encode(f"{username}:{password}".encode())
        token = f'Basic {encoded.decode("utf-8")}'

        response = requests.post(
            "https://libsys.wellcomelibrary.org/iii/sierra-api/v5/token",
            headers={"Authorization": token},
        )

        result = json.loads(response.content.decode("utf-8"))

        return result["access_token"]

    def _parse_response(self, response):
        content = ""
    
        try:
            content = json.loads(response.content)
        except:
            content = response.content
        
        return {
            "status": response.status_code,
            "content": content
        }            
    
    def request_item_for_user(self, patron_id, item_id):
        response = requests.post(
            f"https://libsys.wellcomelibrary.org/iii/sierra-api/v5/patrons/{patron_id}/holds/requests",
            json = {
              "recordType": "i",
              "recordNumber": item_id,
              "pickupLocation": "undefined"
            },
            headers={"Authorization": f"Bearer {self.api_token}"},
        )
        
        return self._parse_response(response)

    def cancel_hold(self, hold_id):               
        response = requests.delete(
            f"https://libsys.wellcomelibrary.org/iii/sierra-api/v5/patrons/holds/{hold_id}",
            headers={"Authorization": f"Bearer {self.api_token}"},
        )
        
        return self._parse_response(response)


    def cancel_all_holds_for_user(self, patron_id):               
        response = requests.delete(
            f"https://libsys.wellcomelibrary.org/iii/sierra-api/v5/patrons/{patron_id}/holds",
            headers={"Authorization": f"Bearer {self.api_token}"},
        )
        
        return self._parse_response(response)
  
    
    def get_holds_for_user(self, patron_id):               
        response = requests.get(
            f"https://libsys.wellcomelibrary.org/iii/sierra-api/v5/patrons/{patron_id}/holds",
            headers={"Authorization": f"Bearer {self.api_token}"},
        )
        
        return self._parse_response(response)
    
    def find_user(self, email):
        find_url = "https://libsys.wellcomelibrary.org/iii/sierra-api/v5/patrons/find"
        response = requests.get(
            f"{find_url}?fields=varFields&varFieldTag=z&varFieldContent={email}",
            headers={"Authorization": f"Bearer {self.api_token}"},
        )

        loaded_response = json.loads(response.content)

        try:
            varFields = loaded_response["varFields"]
        except KeyError:
            return

        username = self._get_field(varFields, "s")
        email    = self._get_field(varFields, "z")

        return {"patronId": loaded_response["id"], "username": username}

In [ ]:
from pprint import pprint
import json


role_arn = "arn:aws:iam::756629837203:role/catalogue-developer"

secrets = AwsSecretsManagerHelper(role_arn)

username = secrets.get_secret('stacks/e2e/test_user/username')
password = secrets.get_secret('stacks/e2e/test_user/password')
client_id = secrets.get_secret('stacks/e2e/test_user/client_id')
api_key = secrets.get_secret('stacks/e2e/test_user/api_key')

sierra_api_key    = secrets.get_secret('stacks/prod/sierra_api_key')
sierra_api_secret = secrets.get_secret('stacks/prod/sierra_api_secret')

stacks_api = StacksApi(role_arn, client_id, api_key, username, password)
sierra_api = SierraApi(sierra_api_key, sierra_api_secret)

response = stacks_api.get_work_cors()

print('')
print('OPTIONS /works/foo')
pprint(response)

response = stacks_api.get_work('x33aa4vt')

print('')
print('GET /works/:id')
pprint(response)

response = stacks_api.request_item('est3d76s')

print('')
print('Check POST /requests')
pprint(response)

response = stacks_api.list_requests()

print('')
print('Check GET /requests')
pprint(response)

response = sierra_api.find_user("digital@wellcomecollection.org")

print('')
print('Sierra get_user_by_email')
pprint(response)
patron_id = response['patronId']

response = sierra_api.get_holds_for_user(patron_id)

print('')
print('Sierra get_holds_for_user')
pprint(response)

response = sierra_api.request_item_for_user(patron_id, 1244886)

print('')
print('Sierra request_item_for_user')
pprint(response)

response = sierra_api.cancel_all_holds_for_user(patron_id)

print('')
print('Sierra cancel_all_holds_for_user')
pprint(response)

response = sierra_api.cancel_hold(151589)

print('')
print('Sierra cancel_hold')
pprint(response)